## **Exercise 18.02**
###  Deploying a Model as a Web API

### Importing modules

In [ ]:
import pandas as pd
import joblib
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

### Loading data

In [ ]:
data =  pd.read_csv('https://raw.githubusercontent.com/fenago/DSBook/main/Chapter%204/glass.csv')

In [ ]:
data.head()

### Extract the 'class_type' response variable using the .pop() method:

In [ ]:

y = data.pop('Type')


In [ ]:
data.fillna(0, inplace=True)

### Dropping "RI"

In [ ]:
X = data.drop('RI', axis=1)

In [ ]:
X.head()

###  Split into training and test sets using the train_test_split function with the 
parameters test_size=0.33 and random_state=888:

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=888)

### Instantiate a RandomForestClassifier with random_state=1 and save it into a new variable called rf_model

In [ ]:
rf_model = RandomForestClassifier(random_state=1)
rf_model.fit(X_train, y_train)

### Predict the outcome for the first record of X_test using the .predict()method

In [ ]:
rf_model.predict([X_test.iloc[0,]])

### Save the RandomForest model as a separate file called model.pkl using joblib.dump()

In [ ]:
joblib.dump(rf_model, "model.pkl")

### Import the socket, threading, requests, json, and numpy packages and the Flask class, as well as the jsonify and request functions from the flask package:

In [ ]:
import socket
import threading
import requests
import json
from flask import Flask, jsonify, request
import numpy as np

In [ ]:
ip_address = socket.gethostbyname(socket.gethostname())
ip_address

### Create a Flask app and save it into a new variable called app

In [ ]:
app = Flask(__name__)

###  Load the pre-trained model using joblib.load():

In [ ]:
trained_model = joblib.load("model.pkl")

### Create an API endpoint for the api path that accepts only POST requests and will call a function called predict().

In [ ]:
@app.route('/api', methods=['POST'])
def predict():
  data = request.get_json()
  prediction = trained_model.predict(data)
  str_pred = np.array2string(prediction)
  return jsonify(str_pred)

### Create a new thread for running your Flask app using the threading.Thread method with the following parameters: target=app.run, kwargs={'host':'0.0.0.0','port':80}:

In [ ]:
flask_thread = threading.Thread(target=app.run, kwargs={'host':'0.0.0.0','port':80})
flask_thread.start()

### Convert the first record of X_test into a list and print its content:

In [ ]:
record = X_test.iloc[0,].to_list()
record

### Create a variable called j_data that will convert this record into JSON by calling the json.dumps() method:

In [ ]:
j_data = json.dumps([record])

### Create a dictionary called headers with the following key-value pairs: 'content-type': 'application/json' and 'Accept-Charset': 'UTF-8'

In [ ]:
headers = {'content-type': 'application/json', 'Accept-Charset': 'UTF-8'}


### Send a HTTP POST request to the server using the requests.post()

In [ ]:
r = requests.post(f"http://{ip_address}/api", data=j_data, 
headers=headers)
r.text